# New York City's Flood Layers Download in Python
Author: Mark Bauer

In [1]:
# importing libraries
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import urllib
import urllib.request
import requests
import zipfile
from zipfile import ZipFile

In [2]:
print('printing packages and versions:\n')

%reload_ext watermark
%watermark -v -p numpy,pandas,geopandas

printing packages and versions:

Python implementation: CPython
Python version       : 3.8.13
IPython version      : 8.4.0

numpy    : 1.23.1
pandas   : 1.4.3
geopandas: 0.11.1



# Sandy Inundation Zone
https://data.cityofnewyork.us/Environment/Sandy-Inundation-Zone/uyj8-7rv5

![sandy-zone](images/sandy-zone.png)

In [3]:
# download url of dataset
url = 'https://data.cityofnewyork.us/api/geospatial/uyj8-7rv5?method=export&format=GeoJSON'
data = 'data/'

fullfilename = os.path.join(data, 'sandy-inundation.geojson')
urllib.request.urlretrieve(url, fullfilename)

('data/sandy-inundation.geojson', <http.client.HTTPMessage at 0x106ba3f70>)

In [4]:
path = 'data/sandy-inundation.geojson'
sandy_gdf = gpd.read_file(path)

print('shape of data: {}'.format(sandy_gdf.shape))
sandy_gdf.head()

shape of data: (492, 8)


,state,sourcedata,id,status,demsource,verified,comments,geometry
0,New York,USGS Mission Assignment/HWM/Surge Sensor Data ...,0.0,Final Field-Verified High Resolution,New York City OEM 1-meter,Yes,Surge extent clipped to 1000ft HSIP buffer,"MULTIPOLYGON (((-74.23650 40.51920, -74.23651 ..."
1,New York,USGS Mission Assignment/HWM/Surge Sensor Data ...,0.0,Final Field-Verified High Resolution,New York City OEM 1-meter,Yes,Surge extent clipped to 1000ft HSIP buffer,"MULTIPOLYGON (((-74.12753 40.54636, -74.12752 ..."
2,New York,USGS Mission Assignment/HWM/Surge Sensor Data ...,0.0,Final Field-Verified High Resolution,New York City OEM 1-meter,Yes,Surge extent clipped to 1000ft HSIP buffer,"MULTIPOLYGON (((-73.91067 40.60621, -73.91068 ..."
3,New York,USGS Mission Assignment/HWM/Surge Sensor Data ...,0.0,Final Field-Verified High Resolution,New York City OEM 1-meter,Yes,Surge extent clipped to 1000ft HSIP buffer,"MULTIPOLYGON (((-74.03144 40.60591, -74.03145 ..."
4,New York,USGS Mission Assignment/HWM/Surge Sensor Data ...,0.0,Final Field-Verified High Resolution,New York City OEM 1-meter,Yes,Surge extent clipped to 1000ft HSIP buffer,"MULTIPOLYGON (((-73.98105 40.60790, -73.98104 ..."


# Hurricane Evacuation Zones
Source: https://data.cityofnewyork.us/Public-Safety/Hurricane-Evacuation-Zones/uihr-hn7s

![hurricane-evac-zones](images/hurricane-evac-zones.png)

In [5]:
# download url of dataset
url = 'https://data.cityofnewyork.us/api/geospatial/uihr-hn7s?method=export&format=GeoJSON'
data = 'data/'

fullfilename = os.path.join(data, 'hurricane-zones.geojson')
urllib.request.urlretrieve(url, fullfilename)

('data/hurricane-zones.geojson', <http.client.HTTPMessage at 0x15fb09be0>)

In [6]:
path = 'data/hurricane-zones.geojson'
hurricanes_gdf = gpd.read_file(path)

print('shape of data: {}'.format(hurricanes_gdf.shape))
hurricanes_gdf.head()

shape of data: (8, 4)


,hurricane_,shape_area,shape_leng,geometry
0,1,1332046368.18,2854091.84974,"MULTIPOLYGON (((-74.21178 40.50819, -74.21179 ..."
1,X,4181913291.16,1719324.71065,"MULTIPOLYGON (((-74.20625 40.51442, -74.20476 ..."
2,0,4740021144.3,2539891.36022,"MULTIPOLYGON (((-73.73838 40.59712, -73.73886 ..."
3,5,532979686.32,1834111.02803,"MULTIPOLYGON (((-74.24969 40.50056, -74.25138 ..."
4,6,541629365.763,2008181.09073,"MULTIPOLYGON (((-74.23826 40.50402, -74.23934 ..."


# Sea Level Rise Maps (2020s 100-year Floodplain)
Source: https://data.cityofnewyork.us/Environment/Sea-Level-Rise-Maps-2020s-100-year-Floodplain-/ezfn-5dsb

![slr-rp100-y00](images/slr-rp100-y00.png)

In [7]:
# download url of dataset
url = 'https://data.cityofnewyork.us/api/geospatial/ezfn-5dsb?method=export&format=GeoJSON'
data = 'data/'

fullfilename = os.path.join(data, 'slr-rp100-y00.geojson')
urllib.request.urlretrieve(url, fullfilename)

('data/slr-rp100-y00.geojson', <http.client.HTTPMessage at 0x16110bdf0>)

In [8]:
path = 'data/slr-rp100-y00.geojson'
slr_rp100_y00 = gpd.read_file(path)

print('shape of data: {}'.format(slr_rp100_y00.shape))
slr_rp100_y00.head()

shape of data: (5982, 11)


,fld_zone,shape_area,join_count,gridcode,shape_leng,static_bfe,id,orig_fid,abfe_0_2pc,target_fid,geometry
0,,0.0,0,900,0.0,0.0,2,0,0,0,"MULTIPOLYGON (((-73.91020 40.91784, -73.91020 ..."
1,,0.0,0,900,0.0,0.0,4,1,0,0,"MULTIPOLYGON (((-73.90993 40.91544, -73.91016 ..."
2,,0.0,0,900,0.0,0.0,5,2,0,0,"MULTIPOLYGON (((-73.91147 40.91345, -73.91147 ..."
3,,0.0,0,1200,0.0,0.0,6,3,0,0,"MULTIPOLYGON (((-73.91165 40.91283, -73.91875 ..."
4,,0.0,0,1200,0.0,0.0,7,4,0,0,"MULTIPOLYGON (((-73.91129 40.91256, -73.91165 ..."


# Sea Level Rise Maps (2020s 500-year Floodplain)
Source: https://data.cityofnewyork.us/Environment/Sea-Level-Rise-Maps-2020s-500-year-Floodplain-/ajyu-7sgg

![slr-rp500-y00](images/slr-rp500-y00.png)

In [9]:
# download url of dataset
url = 'https://data.cityofnewyork.us/api/geospatial/ajyu-7sgg?method=export&format=GeoJSON'
data = 'data/'

fullfilename = os.path.join(data, 'slr-rp500-y00.geojson')
urllib.request.urlretrieve(url, fullfilename)

('data/slr-rp500-y00.geojson', <http.client.HTTPMessage at 0x16110ba90>)

In [10]:
path = 'data/slr-rp500-y00.geojson'
slr_rp500_y00 = gpd.read_file(path)

print('shape of data: {}'.format(slr_rp500_y00.shape))
slr_rp500_y00.head()

shape of data: (4484, 11)


,fld_zone,shape_area,join_count,gridcode,shape_leng,static_bfe,id,orig_fid,abfe_0_2pc,target_fid,geometry
0,VE,206416.636689,2,0,4114.61392959,15.0,0,0,21,1,"MULTIPOLYGON (((-74.24026 40.49848, -74.24049 ..."
1,AE,816603.022555,3,0,8607.97290758,15.0,0,0,19,2,"MULTIPOLYGON (((-74.25101 40.49952, -74.25092 ..."
2,AE,1065737.50301,4,0,11729.3909693,14.0,0,0,19,4,"MULTIPOLYGON (((-74.23614 40.50175, -74.23608 ..."
3,,0.0,0,1800,0.0,0.0,490,193,0,0,"MULTIPOLYGON (((-73.82391 40.84617, -73.82409 ..."
4,AE,633141.40216,1,0,10307.1162296,13.0,0,0,19,3,"MULTIPOLYGON (((-74.24040 40.50129, -74.24032 ..."


# Sea Level Rise Maps (2050s 100-year Floodplain)
Source: https://data.cityofnewyork.us/Environment/Sea-Level-Rise-Maps-2050s-100-year-Floodplain-/hbw8-2bah

![slr-rp100-y30](images/slr-rp100-y30.png)

In [11]:
# download url of dataset
url = 'https://data.cityofnewyork.us/api/geospatial/hbw8-2bah?method=export&format=GeoJSON'
data = 'data/'

fullfilename = os.path.join(data, 'slr-rp100-y30.geojson')
urllib.request.urlretrieve(url, fullfilename)

('data/slr-rp100-y30.geojson', <http.client.HTTPMessage at 0x16110b820>)

In [12]:
path = 'data/slr-rp100-y30.geojson'
slr_rp100_y30 = gpd.read_file(path)

print('shape of data: {}'.format(slr_rp100_y30.shape))
slr_rp100_y30.head()

shape of data: (10611, 8)


,fld_zone,shape_area,join_count,gridcode,shape_leng,static_bfe,abfe_0_2pc,geometry
0,,0.0,0,900,0.0,0.0,0,"MULTIPOLYGON (((-73.90993 40.91544, -73.91000 ..."
1,,0.0,0,900,0.0,0.0,0,"MULTIPOLYGON (((-73.91138 40.91338, -73.91147 ..."
2,,0.0,0,1200,0.0,0.0,0,"MULTIPOLYGON (((-73.91019 40.92018, -73.91126 ..."
3,,0.0,0,900,0.0,0.0,0,"MULTIPOLYGON (((-73.91103 40.91260, -73.91101 ..."
4,,0.0,0,1200,0.0,0.0,0,"MULTIPOLYGON (((-73.91358 40.91144, -73.91368 ..."


# Sea Level Rise Maps (2050s 500-year Floodplain)
Source: https://data.cityofnewyork.us/Environment/Sea-Level-Rise-Maps-2050s-500-year-Floodplain-/qwca-zqw3

![slr-rp500-y30](images/slr-rp500-y30.png)

In [13]:
# download url of dataset
url = 'https://data.cityofnewyork.us/api/geospatial/qwca-zqw3?method=export&format=GeoJSON'
data = 'data/'

fullfilename = os.path.join(data, 'slr-rp500-y30.geojson')
urllib.request.urlretrieve(url, fullfilename)

('data/slr-rp500-y30.geojson', <http.client.HTTPMessage at 0x162e8f9d0>)

In [14]:
path = 'data/slr-rp500-y30.geojson'
slr_rp500_y30 = gpd.read_file(path)

print('shape of data: {}'.format(slr_rp500_y30.shape))
slr_rp500_y30.head()

shape of data: (7055, 8)


,fld_zone,shape_area,join_count,gridcode,shape_leng,static_bfe,abfe_0_2pc,geometry
0,,0.0,0,1500,0.0,0.0,0,"MULTIPOLYGON (((-73.90993 40.91517, -73.91010 ..."
1,,0.0,0,1300,0.0,0.0,0,"MULTIPOLYGON (((-73.88825 40.89957, -73.88801 ..."
2,,0.0,0,1300,0.0,0.0,0,"MULTIPOLYGON (((-73.88852 40.89902, -73.88846 ..."
3,,0.0,0,1300,0.0,0.0,0,"MULTIPOLYGON (((-73.88861 40.89875, -73.88861 ..."
4,,0.0,0,1300,0.0,0.0,0,"MULTIPOLYGON (((-73.88861 40.89861, -73.88874 ..."


# NYC Stormwater Flood Map - Moderate Flood with Current Sea Levels
Source: https://data.cityofnewyork.us/Environment/NYC-Stormwater-Flood-Map-Moderate-Flood-with-Curre/7r5q-vr7p

![stormwater-moderate-y00](images/stormwater-moderate-y00.png)

In [15]:
# download url of dataset
url = 'https://data.cityofnewyork.us/download/7r5q-vr7p/application%2Fx-zip-compressed'
data = 'data-meta/'

fullfilename = os.path.join(data, 'stormwater-y00.zip')
urllib.request.urlretrieve(url, fullfilename)

('data-meta/stormwater-y00.zip', <http.client.HTTPMessage at 0x161658220>)

In [16]:
# a path to our file from our downloads folder
file_path = 'data-meta/stormwater-y00.zip'

# open zipfile and saving items in our zipfolder
items = zipfile.ZipFile(file_path)

# available files in the container
print(items.namelist())
# opening zipfile using 'with' keyword in read mode
with zipfile.ZipFile(file_path, 'r') as file:
    
    # extract all files inside the zip file
    file.extractall(data)

['NYC Stormwater Flood Map - Moderate Flood with Current Sea Levels.gdb/a00000001.gdbindexes', 'NYC Stormwater Flood Map - Moderate Flood with Current Sea Levels.gdb/a00000001.gdbtable', 'NYC Stormwater Flood Map - Moderate Flood with Current Sea Levels.gdb/a00000001.gdbtablx', 'NYC Stormwater Flood Map - Moderate Flood with Current Sea Levels.gdb/a00000001.TablesByName.atx', 'NYC Stormwater Flood Map - Moderate Flood with Current Sea Levels.gdb/a00000002.gdbtable', 'NYC Stormwater Flood Map - Moderate Flood with Current Sea Levels.gdb/a00000002.gdbtablx', 'NYC Stormwater Flood Map - Moderate Flood with Current Sea Levels.gdb/a00000003.gdbindexes', 'NYC Stormwater Flood Map - Moderate Flood with Current Sea Levels.gdb/a00000003.gdbtable', 'NYC Stormwater Flood Map - Moderate Flood with Current Sea Levels.gdb/a00000003.gdbtablx', 'NYC Stormwater Flood Map - Moderate Flood with Current Sea Levels.gdb/a00000004.CatItemsByPhysicalName.atx', 'NYC Stormwater Flood Map - Moderate Flood with C

In [17]:
# read our csv data into a dataframe from our zipfile
file = 'data-meta/NYC Stormwater Flood Map - Moderate Flood with Current Sea Levels.gdb'
stormwater_current = gpd.read_file(file, low_memory=False)

print('shape of data: {}'.format(stormwater_current.shape))
stormwater_current.head()

shape of data: (2, 4)


,Flooding_Category,Shape_Length,Shape_Area,geometry
0,1,6.456233e+06,7.937311e+07,"MULTIPOLYGON Z (((917257.748 120888.120 0.000,..."
1,2,2.601608e+06,4.119771e+07,"MULTIPOLYGON Z (((917182.612 120877.544 0.000,..."


In [18]:
stormwater_current['flood_classification'] = (
    stormwater_current['Flooding_Category']
        .replace({1:'Nuisance Flooding',
                  2:'Deep and Contiguous Flooding',
                  3:'Future High Tides 2080'})
)

stormwater_current.head()

,Flooding_Category,Shape_Length,Shape_Area,geometry,flood_classification
0,1,6.456233e+06,7.937311e+07,"MULTIPOLYGON Z (((917257.748 120888.120 0.000,...",Nuisance Flooding
1,2,2.601608e+06,4.119771e+07,"MULTIPOLYGON Z (((917182.612 120877.544 0.000,...",Deep and Contiguous Flooding


In [19]:
stormwater_current = stormwater_current.explode(index_parts=True).reset_index(drop=True)
stormwater_current['Shape_Area'] = stormwater_current.area 
stormwater_current['Shape_Length'] = stormwater_current.length 

stormwater_current.head()

,Flooding_Category,Shape_Length,Shape_Area,flood_classification,geometry
0,1,269.023026,1479.465601,Nuisance Flooding,"POLYGON Z ((917257.748 120888.120 0.000, 91726..."
1,1,94.018586,347.203105,Nuisance Flooding,"POLYGON Z ((917257.748 120888.120 0.000, 91727..."
2,1,129.191396,658.602805,Nuisance Flooding,"POLYGON Z ((917423.697 120898.496 0.000, 91741..."
3,1,205.237509,1355.404896,Nuisance Flooding,"POLYGON Z ((917337.210 120971.030 0.000, 91734..."
4,1,338.975220,4429.263164,Nuisance Flooding,"POLYGON Z ((917156.306 120935.162 0.000, 91716..."


In [20]:
stormwater_current.to_file('data/stormwater-current.geojson')
gpd.read_file('data/stormwater-current.geojson').head(10)

,Flooding_Category,Shape_Length,Shape_Area,flood_classification,geometry
0,1,269.023026,1479.465601,Nuisance Flooding,"POLYGON Z ((917257.748 120888.120 0.000, 91726..."
1,1,94.018586,347.203105,Nuisance Flooding,"POLYGON Z ((917257.748 120888.120 0.000, 91727..."
2,1,129.191396,658.602805,Nuisance Flooding,"POLYGON Z ((917423.697 120898.496 0.000, 91741..."
3,1,205.237509,1355.404896,Nuisance Flooding,"POLYGON Z ((917337.210 120971.030 0.000, 91734..."
4,1,338.975220,4429.263164,Nuisance Flooding,"POLYGON Z ((917156.306 120935.162 0.000, 91716..."
5,1,14.925456,10.503010,Nuisance Flooding,"POLYGON Z ((915723.364 120997.030 0.000, 91571..."
6,1,11.548442,4.896305,Nuisance Flooding,"POLYGON Z ((915882.411 121050.279 0.000, 91587..."
7,1,50.415989,104.389616,Nuisance Flooding,"POLYGON Z ((917065.591 121125.195 0.000, 91706..."
8,1,481.894051,5532.217354,Nuisance Flooding,"POLYGON Z ((917094.906 121186.003 0.000, 91710..."
9,1,28.154967,30.092303,Nuisance Flooding,"POLYGON Z ((916899.518 121293.338 0.000, 91689..."


# NYC Stormwater Flood Map - Moderate Flood with 2050 Sea Level Rise
Source: https://data.cityofnewyork.us/City-Government/NYC-Stormwater-Flood-Map-Moderate-Flood-with-2050-/5rzh-cyqd

![stormwater-moderate-y30](images/stormwater-moderate-y30.png)

In [21]:
# download url of dataset
url = 'https://data.cityofnewyork.us/download/5rzh-cyqd/application%2Fx-zip-compressed'
data = 'data-meta/'

fullfilename = os.path.join(data, 'stormwater-y30.zip')
urllib.request.urlretrieve(url, fullfilename)

# a path to our file from our downloads folder
file_path = 'data-meta/stormwater-y30.zip'

# open zipfile and saving items in our zipfolder
items = zipfile.ZipFile(file_path)

# available files in the container
print(items.namelist())
# opening zipfile using 'with' keyword in read mode
with zipfile.ZipFile(file_path, 'r') as file:
    
    # extract all files inside the zip file
    file.extractall(data)

['NYC Stormwater Flood Map - Moderate Flood with 2050 Sea Level Rise.gdb/a00000001.freelist', 'NYC Stormwater Flood Map - Moderate Flood with 2050 Sea Level Rise.gdb/a00000001.gdbindexes', 'NYC Stormwater Flood Map - Moderate Flood with 2050 Sea Level Rise.gdb/a00000001.gdbtable', 'NYC Stormwater Flood Map - Moderate Flood with 2050 Sea Level Rise.gdb/a00000001.gdbtablx', 'NYC Stormwater Flood Map - Moderate Flood with 2050 Sea Level Rise.gdb/a00000001.TablesByName.atx', 'NYC Stormwater Flood Map - Moderate Flood with 2050 Sea Level Rise.gdb/a00000002.gdbtable', 'NYC Stormwater Flood Map - Moderate Flood with 2050 Sea Level Rise.gdb/a00000002.gdbtablx', 'NYC Stormwater Flood Map - Moderate Flood with 2050 Sea Level Rise.gdb/a00000003.gdbindexes', 'NYC Stormwater Flood Map - Moderate Flood with 2050 Sea Level Rise.gdb/a00000003.gdbtable', 'NYC Stormwater Flood Map - Moderate Flood with 2050 Sea Level Rise.gdb/a00000003.gdbtablx', 'NYC Stormwater Flood Map - Moderate Flood with 2050 Sea 

In [22]:
# read our csv data into a dataframe from our zipfile
file = 'data-meta/NYC Stormwater Flood Map - Moderate Flood with 2050 Sea Level Rise.gdb'
stormwater_moderate = gpd.read_file(file, low_memory=False)

print('shape of data: {}'.format(stormwater_moderate.shape))
stormwater_moderate.head()

shape of data: (3, 4)


,Flooding_Category,Shape_Length,Shape_Area,geometry
0,1,7.065402e+06,8.657528e+07,"MULTIPOLYGON Z (((917232.601 120841.554 0.000,..."
1,2,2.877719e+06,4.613496e+07,"MULTIPOLYGON Z (((917155.062 120828.741 0.000,..."
2,3,4.994691e+06,1.389420e+08,"MULTIPOLYGON Z (((915689.174 120243.672 0.000,..."


In [23]:
stormwater_moderate['flood_classification'] = (
    stormwater_moderate['Flooding_Category']
        .replace({1:'Nuisance Flooding',
                  2:'Deep and Contiguous Flooding',
                  3:'Future High Tides 2080'})
)

stormwater_moderate.head()

,Flooding_Category,Shape_Length,Shape_Area,geometry,flood_classification
0,1,7.065402e+06,8.657528e+07,"MULTIPOLYGON Z (((917232.601 120841.554 0.000,...",Nuisance Flooding
1,2,2.877719e+06,4.613496e+07,"MULTIPOLYGON Z (((917155.062 120828.741 0.000,...",Deep and Contiguous Flooding
2,3,4.994691e+06,1.389420e+08,"MULTIPOLYGON Z (((915689.174 120243.672 0.000,...",Future High Tides 2080


In [24]:
stormwater_moderate = stormwater_moderate.explode(index_parts=True).reset_index(drop=True)
stormwater_moderate['Shape_Area'] = stormwater_moderate.area 
stormwater_moderate['Shape_Length'] = stormwater_moderate.length 

stormwater_moderate.head()

,Flooding_Category,Shape_Length,Shape_Area,flood_classification,geometry
0,1,23.523243,33.354365,Nuisance Flooding,"POLYGON Z ((917232.601 120841.554 0.000, 91723..."
1,1,36.241793,71.779885,Nuisance Flooding,"POLYGON Z ((917238.644 120854.459 0.000, 91722..."
2,1,138.842343,551.566475,Nuisance Flooding,"POLYGON Z ((917206.327 120862.559 0.000, 91721..."
3,1,104.069124,222.708746,Nuisance Flooding,"POLYGON Z ((917257.748 120888.120 0.000, 91726..."
4,1,94.019239,347.206795,Nuisance Flooding,"POLYGON Z ((917257.748 120888.120 0.000, 91727..."


In [25]:
stormwater_moderate.to_file('data/stormwater-moderate.geojson')
gpd.read_file('data/stormwater-moderate.geojson').head(10)

,Flooding_Category,Shape_Length,Shape_Area,flood_classification,geometry
0,1,23.523243,33.354365,Nuisance Flooding,"POLYGON Z ((917232.601 120841.554 0.000, 91723..."
1,1,36.241793,71.779885,Nuisance Flooding,"POLYGON Z ((917238.644 120854.459 0.000, 91722..."
2,1,138.842343,551.566475,Nuisance Flooding,"POLYGON Z ((917206.327 120862.559 0.000, 91721..."
3,1,104.069124,222.708746,Nuisance Flooding,"POLYGON Z ((917257.748 120888.120 0.000, 91726..."
4,1,94.019239,347.206795,Nuisance Flooding,"POLYGON Z ((917257.748 120888.120 0.000, 91727..."
5,1,129.192222,658.605708,Nuisance Flooding,"POLYGON Z ((917423.698 120898.496 0.000, 91741..."
6,1,205.237414,1355.396943,Nuisance Flooding,"POLYGON Z ((917337.210 120971.030 0.000, 91734..."
7,1,338.975076,4429.266097,Nuisance Flooding,"POLYGON Z ((917156.306 120935.162 0.000, 91716..."
8,1,14.926845,10.504812,Nuisance Flooding,"POLYGON Z ((915723.365 120997.030 0.000, 91571..."
9,1,11.546180,4.894537,Nuisance Flooding,"POLYGON Z ((915882.411 121050.279 0.000, 91587..."


# NYC Stormwater Flood Map - Extreme Flood with 2080 Sea Level Rise
Source: https://data.cityofnewyork.us/City-Government/NYC-Stormwater-Flood-Map-Extreme-Flood-with-2080-S/w8eg-8ha6

![stormwater-extreme-y60](images/stormwater-extreme-y60.png)

In [26]:
# download url of dataset
url = 'https://data.cityofnewyork.us/download/w8eg-8ha6/application%2Fx-zip-compressed'
data = 'data-meta/'

fullfilename = os.path.join(data, 'stormwater-y60.zip')
urllib.request.urlretrieve(url, fullfilename)

# a path to our file from our downloads folder
file_path = 'data-meta/stormwater-y60.zip'

# open zipfile and saving items in our zipfolder
items = zipfile.ZipFile(file_path)

# available files in the container
print(items.namelist())
# opening zipfile using 'with' keyword in read mode
with zipfile.ZipFile(file_path, 'r') as file:
    
    # extract all files inside the zip file
    file.extractall(data)

['NYC Stormwater Flood Map - Extreme Flood with 2080 Sea Level Rise.gdb/a00000001.gdbindexes', 'NYC Stormwater Flood Map - Extreme Flood with 2080 Sea Level Rise.gdb/a00000001.gdbtable', 'NYC Stormwater Flood Map - Extreme Flood with 2080 Sea Level Rise.gdb/a00000001.gdbtablx', 'NYC Stormwater Flood Map - Extreme Flood with 2080 Sea Level Rise.gdb/a00000001.TablesByName.atx', 'NYC Stormwater Flood Map - Extreme Flood with 2080 Sea Level Rise.gdb/a00000002.gdbtable', 'NYC Stormwater Flood Map - Extreme Flood with 2080 Sea Level Rise.gdb/a00000002.gdbtablx', 'NYC Stormwater Flood Map - Extreme Flood with 2080 Sea Level Rise.gdb/a00000003.gdbindexes', 'NYC Stormwater Flood Map - Extreme Flood with 2080 Sea Level Rise.gdb/a00000003.gdbtable', 'NYC Stormwater Flood Map - Extreme Flood with 2080 Sea Level Rise.gdb/a00000003.gdbtablx', 'NYC Stormwater Flood Map - Extreme Flood with 2080 Sea Level Rise.gdb/a00000004.CatItemsByPhysicalName.atx', 'NYC Stormwater Flood Map - Extreme Flood with 20

In [27]:
# read our csv data into a dataframe from our zipfile
file = 'data-meta/NYC Stormwater Flood Map - Extreme Flood with 2080 Sea Level Rise.gdb'
stormwater_extreme = gpd.read_file(file, low_memory=False)

print('shape of data: {}'.format(stormwater_extreme.shape))
stormwater_extreme.head()

shape of data: (3, 4)


,Flooding_Category,Shape_Length,Shape_Area,geometry
0,1,3.264384e+07,3.972152e+08,"MULTIPOLYGON Z (((917206.497 120931.611 0.000,..."
1,2,1.656489e+07,3.054550e+08,"MULTIPOLYGON Z (((917270.790 120873.747 0.000,..."
2,3,8.042525e+06,3.455846e+08,"MULTIPOLYGON Z (((915290.556 120269.389 0.000,..."


In [28]:
stormwater_extreme['flood_classification'] = (
    stormwater_extreme['Flooding_Category']
        .replace({1:'Nuisance Flooding',
                  2:'Deep and Contiguous Flooding',
                  3:'Future High Tides 2080'})
)

stormwater_extreme.head()

,Flooding_Category,Shape_Length,Shape_Area,geometry,flood_classification
0,1,3.264384e+07,3.972152e+08,"MULTIPOLYGON Z (((917206.497 120931.611 0.000,...",Nuisance Flooding
1,2,1.656489e+07,3.054550e+08,"MULTIPOLYGON Z (((917270.790 120873.747 0.000,...",Deep and Contiguous Flooding
2,3,8.042525e+06,3.455846e+08,"MULTIPOLYGON Z (((915290.556 120269.389 0.000,...",Future High Tides 2080


In [29]:
stormwater_extreme = stormwater_extreme.explode(index_parts=True).reset_index(drop=True)
stormwater_extreme['Shape_Area'] = stormwater_extreme.area 
stormwater_extreme['Shape_Length'] = stormwater_extreme.length 

stormwater_extreme.head()

,Flooding_Category,Shape_Length,Shape_Area,flood_classification,geometry
0,1,51.436905,165.359700,Nuisance Flooding,"POLYGON Z ((917206.497 120931.611 0.000, 91720..."
1,1,37.622059,79.460913,Nuisance Flooding,"POLYGON Z ((917309.366 120950.899 0.000, 91730..."
2,1,14.432754,4.925173,Nuisance Flooding,"POLYGON Z ((917309.366 120950.899 0.000, 91730..."
3,1,54.817201,154.010850,Nuisance Flooding,"POLYGON Z ((917277.219 120938.040 0.000, 91726..."
4,1,24.392395,14.030167,Nuisance Flooding,"POLYGON Z ((917311.973 120963.757 0.000, 91730..."


In [30]:
stormwater_extreme.to_file('data/stormwater-extreme.geojson')
gpd.read_file('data/stormwater-extreme.geojson').head(10)

,Flooding_Category,Shape_Length,Shape_Area,flood_classification,geometry
0,1,51.436905,165.359700,Nuisance Flooding,"POLYGON Z ((917206.497 120931.611 0.000, 91720..."
1,1,37.622059,79.460913,Nuisance Flooding,"POLYGON Z ((917309.366 120950.899 0.000, 91730..."
2,1,14.432754,4.925173,Nuisance Flooding,"POLYGON Z ((917309.366 120950.899 0.000, 91730..."
3,1,54.817201,154.010850,Nuisance Flooding,"POLYGON Z ((917277.219 120938.040 0.000, 91726..."
4,1,24.392395,14.030167,Nuisance Flooding,"POLYGON Z ((917311.973 120963.757 0.000, 91730..."
5,1,62.121195,123.414948,Nuisance Flooding,"POLYGON Z ((917206.497 120963.757 0.000, 91720..."
6,1,263.146703,2422.729666,Nuisance Flooding,"POLYGON Z ((917206.497 120931.611 0.000, 91720..."
7,1,29.907432,48.681707,Nuisance Flooding,"POLYGON Z ((917164.130 120979.910 0.000, 91715..."
8,1,116.868920,414.474340,Nuisance Flooding,"POLYGON Z ((917164.130 120979.910 0.000, 91716..."
9,1,50.572910,124.947211,Nuisance Flooding,"POLYGON Z ((915712.807 120994.263 0.000, 91570..."


# New York City Effective Flood Insurance Rate Map (FIRM)
Source: https://msc.fema.gov/portal/advanceSearch

![fema-firm](images/fema-firm.png)

In [31]:
path = 'data-meta/360497_20221122.zip!S_FLD_HAZ_AR.shp'
gdf = gpd.read_file(path)

print(gdf.shape)
gdf.head()

(2147, 20)


,DFIRM_ID,VERSION_ID,FLD_AR_ID,STUDY_TYP,FLD_ZONE,ZONE_SUBTY,SFHA_TF,STATIC_BFE,V_DATUM,DEPTH,LEN_UNIT,VELOCITY,VEL_UNIT,AR_REVERT,AR_SUBTRV,BFE_REVERT,DEP_REVERT,DUAL_ZONE,SOURCE_CIT,geometry
0,360497,1.1.1.0,360497_1,NP,X,AREA OF MINIMAL FLOOD HAZARD,F,-9999.0,None,-9999.0,None,-9999.0,None,None,None,-9999.0,-9999.0,None,360497_FIS1,"POLYGON ((-73.90080 40.56112, -73.90083 40.561..."
1,360497,1.1.1.0,360497_2,NP,X,AREA OF MINIMAL FLOOD HAZARD,F,-9999.0,None,-9999.0,None,-9999.0,None,None,None,-9999.0,-9999.0,None,360497_FIS1,"POLYGON ((-73.83838 40.76860, -73.83847 40.768..."
2,360497,1.1.1.0,360497_3,NP,X,0.2 PCT ANNUAL CHANCE FLOOD HAZARD,F,-9999.0,None,-9999.0,None,-9999.0,None,None,None,-9999.0,-9999.0,None,360497_FIS1,"POLYGON ((-73.79485 40.59911, -73.79482 40.599..."
3,360497,1.1.1.0,360497_4,NP,X,AREA OF MINIMAL FLOOD HAZARD,F,-9999.0,None,-9999.0,None,-9999.0,None,None,None,-9999.0,-9999.0,None,360497_FIS1,"POLYGON ((-73.91605 40.61201, -73.91602 40.612..."
4,360497,1.1.1.0,360497_5,NP,AE,None,T,14.0,NGVD29,-9999.0,Feet,-9999.0,None,None,None,-9999.0,-9999.0,None,360497_FIS1,"POLYGON ((-73.75019 40.78220, -73.75023 40.782..."


In [32]:
gdf.to_file('data/fema-efirm.geojson')
gpd.read_file('data/fema-efirm.geojson').head(10)

,DFIRM_ID,VERSION_ID,FLD_AR_ID,STUDY_TYP,FLD_ZONE,ZONE_SUBTY,SFHA_TF,STATIC_BFE,V_DATUM,DEPTH,LEN_UNIT,VELOCITY,VEL_UNIT,AR_REVERT,AR_SUBTRV,BFE_REVERT,DEP_REVERT,DUAL_ZONE,SOURCE_CIT,geometry
0,360497,1.1.1.0,360497_1,NP,X,AREA OF MINIMAL FLOOD HAZARD,F,-9999.0,None,-9999.0,None,-9999.0,None,None,None,-9999.0,-9999.0,None,360497_FIS1,"POLYGON ((-73.90080 40.56112, -73.90083 40.561..."
1,360497,1.1.1.0,360497_2,NP,X,AREA OF MINIMAL FLOOD HAZARD,F,-9999.0,None,-9999.0,None,-9999.0,None,None,None,-9999.0,-9999.0,None,360497_FIS1,"POLYGON ((-73.83838 40.76860, -73.83847 40.768..."
2,360497,1.1.1.0,360497_3,NP,X,0.2 PCT ANNUAL CHANCE FLOOD HAZARD,F,-9999.0,None,-9999.0,None,-9999.0,None,None,None,-9999.0,-9999.0,None,360497_FIS1,"POLYGON ((-73.79485 40.59911, -73.79482 40.599..."
3,360497,1.1.1.0,360497_4,NP,X,AREA OF MINIMAL FLOOD HAZARD,F,-9999.0,None,-9999.0,None,-9999.0,None,None,None,-9999.0,-9999.0,None,360497_FIS1,"POLYGON ((-73.91605 40.61201, -73.91602 40.612..."
4,360497,1.1.1.0,360497_5,NP,AE,None,T,14.0,NGVD29,-9999.0,Feet,-9999.0,None,None,None,-9999.0,-9999.0,None,360497_FIS1,"POLYGON ((-73.75019 40.78220, -73.75023 40.782..."
5,360497,1.1.1.0,360497_6,NP,X,0.2 PCT ANNUAL CHANCE FLOOD HAZARD,F,-9999.0,None,-9999.0,None,-9999.0,None,None,None,-9999.0,-9999.0,None,360497_FIS1,"POLYGON ((-73.91628 40.55918, -73.91642 40.559..."
6,360497,1.1.1.0,360497_7,NP,AE,None,T,10.0,NGVD29,-9999.0,Feet,-9999.0,None,None,None,-9999.0,-9999.0,None,360497_FIS1,"POLYGON ((-73.99721 40.58246, -73.99731 40.582..."
7,360497,1.1.1.0,360497_8,NP,X,0.2 PCT ANNUAL CHANCE FLOOD HAZARD,F,-9999.0,None,-9999.0,None,-9999.0,None,None,None,-9999.0,-9999.0,None,360497_FIS1,"POLYGON ((-73.93169 40.57567, -73.93173 40.575..."
8,360497,1.1.1.0,360497_9,NP,X,0.2 PCT ANNUAL CHANCE FLOOD HAZARD,F,-9999.0,None,-9999.0,None,-9999.0,None,None,None,-9999.0,-9999.0,None,360497_FIS1,"POLYGON ((-74.22601 40.51211, -74.22609 40.512..."
9,360497,1.1.1.0,360497_10,NP,X,0.2 PCT ANNUAL CHANCE FLOOD HAZARD,F,-9999.0,None,-9999.0,None,-9999.0,None,None,None,-9999.0,-9999.0,None,360497_FIS1,"POLYGON ((-73.91048 40.80230, -73.91052 40.802..."


# New York City Preliminary Flood Insurance Rate Map (PFIRM)
Source: https://r2-coastal-fema.hub.arcgis.com/pages/preliminary-flood-map-data

![fema-pfirm](images/fema-pfirm.png)

In [33]:
# url path from the web
url = 'https://msc.fema.gov/portal/downloadProduct?productID=360497_PRELIMDB'
data = 'data-meta/'

fullfilename = os.path.join(data, 'fema-pfirm.zip')
urllib.request.urlretrieve(url, fullfilename)

# a path to our file from our downloads folder
file_path = 'data-meta/fema-pfirm.zip'

# open zipfile and saving items in our zipfolder
items = zipfile.ZipFile(file_path)

# available files in the container
print(items.namelist())
# opening zipfile using 'with' keyword in read mode
with zipfile.ZipFile(file_path, 'r') as file:
    
    # extract all files inside the zip file
    file.extractall(data)

['360497_PRELIM_metadata.txt', '360497_PRELIM_metadata.xml', 'l_comm_info.dbf', 'L_PAN_REVIS.dbf', 'L_POL_FHBM.dbf', 'l_stn_start.dbf', 'study_info.dbf', 's_base_index.dbf', 'S_BASE_INDEX.prj', 's_base_index.shp', 's_base_index.shx', 's_bfe.dbf', 'S_BFE.prj', 's_bfe.shp', 's_bfe.shx', 's_cbrs.dbf', 'S_CBRS.prj', 's_cbrs.shp', 's_cbrs.shx', 'S_CST_TSCT_LN.dbf', 'S_CST_TSCT_LN.prj', 'S_CST_TSCT_LN.shp', 'S_CST_TSCT_LN.shx', 's_firm_pan.dbf', 'S_FIRM_PAN.prj', 's_firm_pan.shp', 's_firm_pan.shx', 's_fld_haz_ar.dbf', 'S_FLD_HAZ_AR.prj', 's_fld_haz_ar.shp', 's_fld_haz_ar.shx', 's_fld_haz_ln.dbf', 'S_FLD_HAZ_LN.prj', 's_fld_haz_ln.shp', 's_fld_haz_ln.shx', 's_gen_struct.dbf', 'S_GEN_STRUCT.prj', 's_gen_struct.shp', 's_gen_struct.shx', 's_label_ld.dbf', 's_label_ld.prj', 's_label_ld.sbn', 's_label_ld.sbx', 's_label_ld.shp', 's_label_ld.shx', 's_label_pt.dbf', 's_label_pt.prj', 's_label_pt.sbn', 's_label_pt.sbx', 's_label_pt.shp', 's_label_pt.shx', 'S_LiMWA.dbf', 'S_LiMWA.prj', 'S_LiMWA.shp', '

In [34]:
# reading in shape file
path = 'data-meta/s_fld_haz_ar.shp'
pfirm_df = gpd.read_file(path)

# previewing first five rows of data
pfirm_df.head()

,FLD_AR_ID,FLD_ZONE,FLOODWAY,SFHA_TF,STATIC_BFE,V_DATUM,DEPTH,LEN_UNIT,VELOCITY,VEL_UNIT,AR_REVERT,BFE_REVERT,DEP_REVERT,SOURCE_CIT,geometry
0,81,AE,None,T,13.0,NAVD88,-9999.0,FEET,-9999.0,None,None,-9999.0,-9999.0,STUDY4,"POLYGON ((1032001.691 263867.839, 1032006.047 ..."
1,82,X,None,F,-9999.0,None,-9999.0,None,-9999.0,None,None,-9999.0,-9999.0,STUDY4,"POLYGON ((1033748.952 263778.464, 1033752.656 ..."
2,83,0.2 PCT ANNUAL CHANCE FLOOD HAZARD,None,F,-9999.0,None,-9999.0,None,-9999.0,None,None,-9999.0,-9999.0,STUDY4,"POLYGON ((1033309.701 263544.181, 1033322.210 ..."
3,84,0.2 PCT ANNUAL CHANCE FLOOD HAZARD,None,F,-9999.0,None,-9999.0,None,-9999.0,None,None,-9999.0,-9999.0,STUDY4,"POLYGON ((1033414.148 262658.868, 1033414.934 ..."
4,85,0.2 PCT ANNUAL CHANCE FLOOD HAZARD,None,F,-9999.0,None,-9999.0,None,-9999.0,None,None,-9999.0,-9999.0,STUDY4,"POLYGON ((1034089.115 262466.034, 1034089.402 ..."


In [35]:
pfirm_df.to_file('data/fema-pfirm.geojson')
gpd.read_file('data/fema-pfirm.geojson').head(10)

,FLD_AR_ID,FLD_ZONE,FLOODWAY,SFHA_TF,STATIC_BFE,V_DATUM,DEPTH,LEN_UNIT,VELOCITY,VEL_UNIT,AR_REVERT,BFE_REVERT,DEP_REVERT,SOURCE_CIT,geometry
0,81,AE,None,T,13.0,NAVD88,-9999.0,FEET,-9999.0,None,None,-9999.0,-9999.0,STUDY4,"POLYGON ((1032001.691 263867.839, 1032006.047 ..."
1,82,X,None,F,-9999.0,None,-9999.0,None,-9999.0,None,None,-9999.0,-9999.0,STUDY4,"POLYGON ((1033748.952 263778.464, 1033752.656 ..."
2,83,0.2 PCT ANNUAL CHANCE FLOOD HAZARD,None,F,-9999.0,None,-9999.0,None,-9999.0,None,None,-9999.0,-9999.0,STUDY4,"POLYGON ((1033309.701 263544.181, 1033322.210 ..."
3,84,0.2 PCT ANNUAL CHANCE FLOOD HAZARD,None,F,-9999.0,None,-9999.0,None,-9999.0,None,None,-9999.0,-9999.0,STUDY4,"POLYGON ((1033414.148 262658.868, 1033414.934 ..."
4,85,0.2 PCT ANNUAL CHANCE FLOOD HAZARD,None,F,-9999.0,None,-9999.0,None,-9999.0,None,None,-9999.0,-9999.0,STUDY4,"POLYGON ((1034089.115 262466.034, 1034089.402 ..."
5,86,0.2 PCT ANNUAL CHANCE FLOOD HAZARD,None,F,-9999.0,None,-9999.0,None,-9999.0,None,None,-9999.0,-9999.0,STUDY4,"POLYGON ((1033316.808 262437.816, 1033316.108 ..."
6,87,X,None,F,-9999.0,None,-9999.0,None,-9999.0,None,None,-9999.0,-9999.0,STUDY4,"POLYGON ((1034013.168 257011.473, 1034007.732 ..."
7,88,AE,None,T,15.0,NAVD88,-9999.0,FEET,-9999.0,None,None,-9999.0,-9999.0,STUDY4,"POLYGON ((1036695.145 256698.584, 1036706.934 ..."
8,89,0.2 PCT ANNUAL CHANCE FLOOD HAZARD,None,F,-9999.0,None,-9999.0,None,-9999.0,None,None,-9999.0,-9999.0,STUDY4,"POLYGON ((1034238.403 256848.609, 1034238.403 ..."
9,90,0.2 PCT ANNUAL CHANCE FLOOD HAZARD,None,F,-9999.0,None,-9999.0,None,-9999.0,None,None,-9999.0,-9999.0,STUDY4,"POLYGON ((1010028.659 257077.165, 1010048.110 ..."


In [36]:
# files in data folder
%ls data/

fema-efirm.geojson           slr-rp500-y00.geojson
fema-pfirm.geojson           slr-rp500-y30.geojson
hurricane-zones.geojson      stormwater-current.geojson
sandy-inundation.geojson     stormwater-extreme.geojson
slr-rp100-y00.geojson        stormwater-moderate.geojson
slr-rp100-y30.geojson
